## HF config

In [1]:
import os
import dotenv

%cd /home/code/NeuraLens
# Load environment variables from .env file
dotenv.load_dotenv(".env")
# Access the HF_TOKEN environment variable
HF_TOKEN = os.getenv("HF_TOKEN")

/home/code/NeuraLens


/opt/conda/envs/medgemma/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig

model_id = "google/medgemma-4b-it"

# Check if GPU supports bfloat16
if torch.cuda.get_device_capability()[0] < 8:
    raise ValueError(
        "GPU does not support bfloat16, please use a GPU that supports bfloat16."
    )

model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=model_kwargs["torch_dtype"],
    bnb_4bit_quant_storage=model_kwargs["torch_dtype"],
)

base_model = AutoModelForImageTextToText.from_pretrained(model_id, **model_kwargs)
processor = AutoProcessor.from_pretrained(model_id)

# Use right padding to avoid issues during training
processor.tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
# from datasets import Dataset as HFDataset, DatasetDict
# from torch.utils.data import DataLoader
# from PIL import Image
# from torchvision import transforms
# import numpy as np
# from pathlib import Path
# import torch
# from sklearn.model_selection import train_test_split
# from typing import Any


# # TODO add multiprocessing to speed up image loading
# class tissue_dataset:
#     def __init__(
#         self,
#         images_dir_path="src/dataset/Motic-Human-tissues",
#         transform=None,
#         split_ratio=0.8,
#         train_batch_size=8,
#         val_batch_size=8,
#     ):
#         """Initializes the tissue dataset.
#         Args:
#             images_dir_path (str): Path to the directory containing tissue images.
#             transform (callable, optional): A function/transform that takes in an image and returns a transformed version.
#             split_ratio (float): The ratio of the dataset to be used for training.
#             train_batch_size (int): The number of samples per batch for training.
#             val_batch_size (int): The number of samples per batch for validation.
#         """
#         self.split_ratio = split_ratio
#         self.train_batch_size = train_batch_size
#         self.val_batch_size = val_batch_size

#         images = []
#         focus = []
#         type_labels = []
#         zoom = []

#         images_dir = Path(images_dir_path)
#         for image in images_dir.glob("*.jpg"):
#             image_name = image.name.lower()
#             if image_name.find("calibration") == -1:
#                 images.append(str(image))

#                 if image_name.find("focus") != -1:
#                     focus.append(0)
#                 else:
#                     focus.append(1)

#                 position = image_name.find("4x")
#                 if position != -1:
#                     zoom.append(4)
#                 else:
#                     position = image_name.find("10x")
#                     if position != -1:
#                         zoom.append(10)
#                     else:
#                         position = image_name.find("10x")
#                         if position != -1:
#                             zoom.append(10)
#                         else:
#                             position = image_name.find("20x")
#                             if position != -1:
#                                 zoom.append(20)
#                             else:
#                                 position = image_name.find("40x")
#                                 if position != -1:
#                                     zoom.append(40)

#                 type_labels.append(image_name[: position - 1].split("-")[0])
#         print(f"Found {len(images)} images in {images_dir_path}")

#         if transform is not None:
#             self.transform = transform
#         else:
#             self.transform = transforms.ToTensor()

#         self.zoom_classes = sorted(list(set(zoom)))  # e.g., [4, 10, 20, 40]
#         self.type_classes = sorted(
#             list(set(type_labels))
#         )  # e.g., ['adipose', 'blood', 'bone', etc.]
#         self.focus_classes = [0, 1]  # Focused (0) or Unfocused (1)

#         # Create mappings
#         self.zoom_to_idx = {
#             zoom_val: idx for idx, zoom_val in enumerate(self.zoom_classes)
#         }
#         self.type_to_idx = {
#             type_name: idx for idx, type_name in enumerate(self.type_classes)
#         }
#         # Focused (0) or Unfocused (1)
#         self.focus_to_idx = {
#             "focused": 0,
#             "unfocused": 1,
#         }

#         # Convert zoom and type to indices
#         zoom_indices = [self.zoom_to_idx[z] for z in zoom]
#         type_indices = [self.type_to_idx[t] for t in type_labels]

#         # Split data before creating HuggingFace datasets
#         self._split_data(images, focus, zoom_indices, type_indices)

#         type_classes_mapping = {v: k for k, v in self.type_to_idx.items()}

#         self.options = "\n".join(
#             [f"{i}: {type_classes_mapping[i]}" for i in type_classes_mapping]
#         )

#         self.PROMPT = f"What is the most likely body part type shown in the histopathology image?\n{self.options}"

#         # Create HuggingFace DatasetDict
#         self._create_hf_datasets()

#     def _split_data(self, images, focus, zoom_indices, type_indices):
#         """Split the data into train and validation sets."""
#         # Split the indices without stratification
#         train_indices, val_indices = train_test_split(
#             range(len(images)), test_size=1.0 - self.split_ratio, random_state=42
#         )

#         # Create train split
#         self.train_data = {
#             "image_path": [images[i] for i in train_indices],
#             "focus": [focus[i] for i in train_indices],
#             "zoom": [zoom_indices[i] for i in train_indices],
#             "type": [type_indices[i] for i in train_indices],
#         }

#         # Create validation split
#         self.val_data = {
#             "image_path": [images[i] for i in val_indices],
#             "focus": [focus[i] for i in val_indices],
#             "zoom": [zoom_indices[i] for i in val_indices],
#             "type": [type_indices[i] for i in val_indices],
#         }

#     def _create_hf_datasets(self):
#         """Create HuggingFace DatasetDict with train and validation splits."""
#         # Create individual datasets
#         print("Creating HuggingFace datasets...")
#         train_dataset = HFDataset.from_dict(self.train_data)
#         val_dataset = HFDataset.from_dict(self.val_data)

#         # Apply message formatting for chat-based training
#         print("Formatting training data...")
#         train_dataset = train_dataset.map(self._format_data, batched=False)

#         print("Formatting validation data...")
#         val_dataset = val_dataset.map(self._format_test_data, batched=False)

#         # Create DatasetDict
#         self.dataset = DatasetDict({"train": train_dataset, "validation": val_dataset})

#     def build_train_val_loaders(self) -> None:
#         """Create DataLoaders for training and validation."""
#         self.train_dataset = self.dataset["train"]
#         self.val_dataset = self.dataset["validation"]

#         # Set format for PyTorch compatibility
#         self.train_dataset.set_format("torch")
#         self.val_dataset.set_format("torch")

#         # Create DataLoaders
#         self.train_loader = DataLoader(
#             self.train_dataset,
#             batch_size=self.train_batch_size,
#             shuffle=True,
#             collate_fn=self._collate_fn,
#         )
#         self.val_loader = DataLoader(
#             self.val_dataset,
#             batch_size=self.val_batch_size,
#             collate_fn=self._collate_fn,
#         )

#     def _collate_fn(self, examples: list[dict[str, Any]]):
#         texts = []
#         images = []
#         for example in examples:
#             image = Image.open(example["image_path"]).convert("RGB")
#             images.append(image)
#             # print(f"Image: {example['image_path']}, size: {image.size}")
#             text = self.processor.apply_chat_template(
#                 example["messages"], add_generation_prompt=False, tokenize=False
#             ).strip()
#             # print(f"Text: {text}, length: {len(text)}")
#             texts.append(text)

#         # print(f"Number of images: {len(images)}, Number of texts: {len(texts)}")

#         # Process each image-text pair individually and then batch them
#         batched_inputs = []
#         for i in range(len(images)):
#             # Process one image-text pair at a time
#             single_batch = self.processor(
#                 text=[texts[i]], images=[images[i]], return_tensors="pt", padding=True
#             )
#             batched_inputs.append(single_batch)

#         # Now manually batch the results
#         batch = {}
#         for key in batched_inputs[0].keys():
#             if key == "pixel_values":
#                 # Stack image tensors
#                 batch[key] = torch.cat([b[key] for b in batched_inputs], dim=0)
#             else:
#                 # Pad and stack text tensors
#                 max_length = max(b[key].shape[1] for b in batched_inputs)
#                 padded_tensors = []
#                 for b in batched_inputs:
#                     tensor = b[key]
#                     if tensor.shape[1] < max_length:
#                         # Pad with tokenizer's pad_token_id
#                         padding = torch.full(
#                             (tensor.shape[0], max_length - tensor.shape[1]),
#                             self.processor.tokenizer.pad_token_id,
#                             dtype=tensor.dtype,
#                         )
#                         tensor = torch.cat([tensor, padding], dim=1)
#                     padded_tensors.append(tensor)
#                 batch[key] = torch.cat(padded_tensors, dim=0)

#         # The labels are the input_ids, with the padding and image tokens masked in
#         # the loss computation
#         labels = batch["input_ids"].clone()

#         # Mask image tokens
#         image_token_id = self.processor.tokenizer.convert_tokens_to_ids(
#             self.processor.tokenizer.special_tokens_map["boi_token"]
#         )

#         # Mask tokens that are not used in the loss computation
#         labels[labels == self.processor.tokenizer.pad_token_id] = -100
#         labels[labels == image_token_id] = -100
#         labels[labels == 262144] = -100

#         batch["labels"] = labels
#         return batch

#     def set_processor(self, processor):
#         """Set the processor for the collate function."""
#         self.processor = processor

#     def _format_data(self, example: dict[str, Any]) -> dict[str, Any]:
#         """Format data for chat-based training."""
#         # Get the tissue type label and convert to class name
#         tissue_type_idx = example["type"]
#         tissue_type_name = self.type_classes[tissue_type_idx]

#         example["messages"] = [
#             {
#                 "role": "user",
#                 "content": [
#                     {
#                         "type": "image",
#                     },
#                     {
#                         "type": "text",
#                         "text": self.PROMPT,
#                     },
#                 ],
#             },
#             {
#                 "role": "assistant",
#                 "content": [
#                     {
#                         "type": "text",
#                         "text": f"{tissue_type_idx}: {tissue_type_name}",
#                     },
#                 ],
#             },
#         ]
#         return example

#     def _format_test_data(self, example: dict[str, Any]) -> dict[str, Any]:
#         """Format data for chat-based training."""
#         # Get the tissue type label and convert to class name

#         example["messages"] = [
#             {
#                 "role": "user",
#                 "content": [
#                     {
#                         "type": "image",
#                     },
#                     {
#                         "type": "text",
#                         "text": self.PROMPT,
#                     },
#                 ],
#             },
#         ]
#         return example

#     def postprocess(
#         self, prediction: list[dict[str, str]], do_full_match: bool = False
#     ) -> int:
#         response_text = prediction[0]["generated_text"]
#         if do_full_match:
#             # Try to extract the number directly from the response
#             try:
#                 return int(response_text.strip())
#             except ValueError:
#                 pass

#         for i, label in enumerate(self.type_classes):
#             # Search for `X: tissue type` or `(X) tissue type` in the response
#             if (
#                 f"{i}: {label}" in response_text
#                 or f"({i}) {label}" in response_text
#                 or label in response_text
#             ):
#                 return i
#         return -1

#     def __len__(self):
#         return len(self.dataset["train"]) + len(self.dataset["validation"])

#     def get_num_classes(self):
#         """Returns the number of classes for each label type."""
#         return {
#             "focus": 2,  # focused (0) or unfocused (1)
#             "zoom": len(self.zoom_classes),
#             "type": len(self.type_classes),
#         }

In [4]:
from datasets import Dataset as HFDataset, DatasetDict
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms
import numpy as np
from pathlib import Path
import torch
from sklearn.model_selection import train_test_split
from typing import Any


# TODO add multiprocessing to speed up image loading
class tissue_dataset:
    def __init__(
        self,
        images_dir_path="src/dataset/Motic-Human-tissues",
        transform=None,
        split_ratio=0.8,
        train_batch_size=8,
        val_batch_size=8,
    ):
        """Initializes the tissue dataset.
        Args:
            images_dir_path (str): Path to the directory containing tissue images.
            transform (callable, optional): A function/transform that takes in an image and returns a transformed version.
            split_ratio (float): The ratio of the dataset to be used for training.
            train_batch_size (int): The number of samples per batch for training.
            val_batch_size (int): The number of samples per batch for validation.
        """
        self.split_ratio = split_ratio
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size

        images = []
        focus = []
        type_labels = []
        zoom = []

        images_dir = Path(images_dir_path)
        for image in images_dir.glob("*.jpg"):
            image_name = image.name.lower()
            if image_name.find("calibration") == -1:
                images.append(str(image))

                if image_name.find("focus") != -1:
                    focus.append(0)
                else:
                    focus.append(1)

                position = image_name.find("4x")
                if position != -1:
                    zoom.append(4)
                else:
                    position = image_name.find("10x")
                    if position != -1:
                        zoom.append(10)
                    else:
                        position = image_name.find("10x")
                        if position != -1:
                            zoom.append(10)
                        else:
                            position = image_name.find("20x")
                            if position != -1:
                                zoom.append(20)
                            else:
                                position = image_name.find("40x")
                                if position != -1:
                                    zoom.append(40)

                type_labels.append(image_name[: position - 1].split("-")[0])
        print(f"Found {len(images)} images in {images_dir_path}")

        if transform is not None:
            self.transform = transform
        else:
            self.transform = transforms.ToTensor()

        self.zoom_classes = sorted(list(set(zoom)))  # e.g., [4, 10, 20, 40]
        self.type_classes = sorted(
            list(set(type_labels))
        )  # e.g., ['adipose', 'blood', 'bone', etc.]
        self.focus_classes = [0, 1]  # Focused (0) or Unfocused (1)

        # Create mappings
        self.zoom_to_idx = {
            zoom_val: idx for idx, zoom_val in enumerate(self.zoom_classes)
        }
        self.type_to_idx = {
            type_name: idx for idx, type_name in enumerate(self.type_classes)
        }

        # Convert zoom and type to indices
        zoom_indices = [self.zoom_to_idx[z] for z in zoom]
        type_indices = [self.type_to_idx[t] for t in type_labels]

        # Split data before creating HuggingFace datasets
        self._split_data(images, focus, zoom_indices, type_indices)

        type_classes_mapping = {v: k for k, v in self.type_to_idx.items()}
        zoom_classes_mapping = {v: k for k, v in self.zoom_to_idx.items()}
        focus_classes_mapping = {0: "focused", 1: "unfocused"}

        self.type_options = "\n".join(
            [f"{i}: {type_classes_mapping[i]}" for i in type_classes_mapping]
        )
        self.zoom_options = "\n".join(
            [f"{i}: {zoom_classes_mapping[i]}x" for i in zoom_classes_mapping]
        )
        self.focus_options = "\n".join(
            [f"{i}: {focus_classes_mapping[i]}" for i in focus_classes_mapping]
        )

        self.PROMPT = f"""Analyze this histopathology image and provide the following information:

        Tissue Type:
        {self.type_options}

        Zoom Level:
        {self.zoom_options}

        Focus Quality:
        {self.focus_options}

        Please respond in the following JSON format:
        {{
        "tissue_type": "X: tissue_name",
        "zoom_level": "Y: Zx",
        "focus_quality": "Z: focus_status"
        }}"""
        # Create HuggingFace DatasetDict
        self._create_hf_datasets()

    def _split_data(self, images, focus, zoom_indices, type_indices):
        """Split the data into train and validation sets."""
        # Split the indices without stratification
        train_indices, val_indices = train_test_split(
            range(len(images)), test_size=1.0 - self.split_ratio, random_state=42
        )

        # Create train split
        self.train_data = {
            "image_path": [images[i] for i in train_indices],
            "focus": [focus[i] for i in train_indices],
            "zoom": [zoom_indices[i] for i in train_indices],
            "type": [type_indices[i] for i in train_indices],
        }

        # Create validation split
        self.val_data = {
            "image_path": [images[i] for i in val_indices],
            "focus": [focus[i] for i in val_indices],
            "zoom": [zoom_indices[i] for i in val_indices],
            "type": [type_indices[i] for i in val_indices],
        }

    def _create_hf_datasets(self):
        """Create HuggingFace DatasetDict with train and validation splits."""
        # Create individual datasets
        print("Creating HuggingFace datasets...")
        train_dataset = HFDataset.from_dict(self.train_data)
        val_dataset = HFDataset.from_dict(self.val_data)

        # Apply message formatting for chat-based training
        print("Formatting training data...")
        train_dataset = train_dataset.map(self._format_data, batched=False)

        print("Formatting validation data...")
        val_dataset = val_dataset.map(self._format_test_data, batched=False)

        # Create DatasetDict
        self.dataset = DatasetDict({"train": train_dataset, "validation": val_dataset})

    def build_train_val_loaders(self) -> None:
        """Create DataLoaders for training and validation."""
        self.train_dataset = self.dataset["train"]
        self.val_dataset = self.dataset["validation"]

        # Set format for PyTorch compatibility
        self.train_dataset.set_format("torch")
        self.val_dataset.set_format("torch")

        # Create DataLoaders
        self.train_loader = DataLoader(
            self.train_dataset,
            batch_size=self.train_batch_size,
            shuffle=True,
            collate_fn=self._collate_fn,
        )
        self.val_loader = DataLoader(
            self.val_dataset,
            batch_size=self.val_batch_size,
            collate_fn=self._collate_fn,
        )

    def _collate_fn(self, examples: list[dict[str, Any]]):
        texts = []
        images = []
        for example in examples:
            image = Image.open(example["image_path"]).convert("RGB")
            images.append(image)
            # print(f"Image: {example['image_path']}, size: {image.size}")
            text = self.processor.apply_chat_template(
                example["messages"], add_generation_prompt=False, tokenize=False
            ).strip()
            # print(f"Text: {text}, length: {len(text)}")
            texts.append(text)

        # print(f"Number of images: {len(images)}, Number of texts: {len(texts)}")

        # Process each image-text pair individually and then batch them
        batched_inputs = []
        for i in range(len(images)):
            # Process one image-text pair at a time
            single_batch = self.processor(
                text=[texts[i]], images=[images[i]], return_tensors="pt", padding=True
            )
            batched_inputs.append(single_batch)

        # Now manually batch the results
        batch = {}
        for key in batched_inputs[0].keys():
            if key == "pixel_values":
                # Stack image tensors
                batch[key] = torch.cat([b[key] for b in batched_inputs], dim=0)
            else:
                # Pad and stack text tensors
                max_length = max(b[key].shape[1] for b in batched_inputs)
                padded_tensors = []
                for b in batched_inputs:
                    tensor = b[key]
                    if tensor.shape[1] < max_length:
                        # Pad with tokenizer's pad_token_id
                        padding = torch.full(
                            (tensor.shape[0], max_length - tensor.shape[1]),
                            self.processor.tokenizer.pad_token_id,
                            dtype=tensor.dtype,
                        )
                        tensor = torch.cat([tensor, padding], dim=1)
                    padded_tensors.append(tensor)
                batch[key] = torch.cat(padded_tensors, dim=0)

        # The labels are the input_ids, with the padding and image tokens masked in
        # the loss computation
        labels = batch["input_ids"].clone()

        # Mask image tokens
        image_token_id = self.processor.tokenizer.convert_tokens_to_ids(
            self.processor.tokenizer.special_tokens_map["boi_token"]
        )

        # Mask tokens that are not used in the loss computation
        labels[labels == self.processor.tokenizer.pad_token_id] = -100
        labels[labels == image_token_id] = -100
        labels[labels == 262144] = -100

        batch["labels"] = labels
        return batch

    def set_processor(self, processor):
        """Set the processor for the collate function."""
        self.processor = processor

    def _format_data(self, example: dict[str, Any]) -> dict[str, Any]:
        """Format data for chat-based training."""
        tissue_type_idx = example["type"]
        tissue_type_name = self.type_classes[tissue_type_idx]
        zoom_idx = example["zoom"]
        zoom_value = self.zoom_classes[zoom_idx]
        focus_idx = example["focus"]
        focus_status = "focused" if focus_idx == 0 else "unfocused"

        # Randomly choose between JSON and natural language responses (70% JSON, 30% natural)
        import random

        use_json = random.random() < 0.7

        if use_json:
            response_text = f"""{{
    "tissue_type": "{tissue_type_idx}: {tissue_type_name}",
    "zoom_level": "{zoom_idx}: {zoom_value}x",
    "focus_quality": "{focus_idx}: {focus_status}"
    }}"""
        else:
            response_text = f"This histopathology image shows {tissue_type_name} tissue at {zoom_value}x magnification. The image appears to be {focus_status}."

        example["messages"] = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": self.PROMPT},
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": response_text},
                ],
            },
        ]
        return example

    def _format_test_data(self, example: dict[str, Any]) -> dict[str, Any]:
        """Format data for chat-based training."""
        example["messages"] = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                    },
                    {
                        "type": "text",
                        "text": self.PROMPT,
                    },
                ],
            },
        ]
        return example

    def postprocess(
        self, prediction: list[dict[str, str]], do_full_match: bool = False
    ) -> dict:
        response_text = prediction[0]["generated_text"]

        # Clean the response text first
        response_text = response_text.strip()

        # Try to extract the first JSON block
        import re
        import json

        # Look for JSON pattern
        json_pattern = r"\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}"
        json_matches = re.findall(json_pattern, response_text)

        for json_match in json_matches:
            try:
                parsed = json.loads(json_match)
                if all(
                    key in parsed
                    for key in ["tissue_type", "zoom_level", "focus_quality"]
                ):
                    return {
                        "tissue_type": self._extract_index(
                            parsed.get("tissue_type", "")
                        ),
                        "zoom_level": self._extract_index(parsed.get("zoom_level", "")),
                        "focus_quality": self._extract_index(
                            parsed.get("focus_quality", "")
                        ),
                    }
            except (json.JSONDecodeError, KeyError):
                continue

        # Fallback to text parsing if no valid JSON found
        return {
            "tissue_type": self._extract_tissue_from_text(response_text),
            "zoom_level": self._extract_zoom_from_text(response_text),
            "focus_quality": self._extract_focus_from_text(response_text),
        }

    def _extract_index(self, text: str) -> int:
        """Extract index from 'X: value' format"""
        try:
            return int(text.split(":")[0].strip())
        except (ValueError, IndexError):
            return -1

    def __len__(self):
        return len(self.dataset["train"]) + len(self.dataset["validation"])

    def get_num_classes(self):
        """Returns the number of classes for each label type."""
        return {
            "focus": 2,  # focused (0) or unfocused (1)
            "zoom": len(self.zoom_classes),
            "type": len(self.type_classes),
        }

## Dataset preprocess

In [5]:
# assuming dataset is already downloaded in src/dataset
# from src.dataset import tissue_dataset
from torchvision import transforms

dataset_path = "src/dataset/Motic-Human-tissues"
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize((512, 512)),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])
transform = None

train_batch_size = 1
val_batch_size = 8
split_ratio = 0.8

dataset = tissue_dataset(
    images_dir_path=dataset_path,
    transform=transform,
    split_ratio=split_ratio,
    train_batch_size=train_batch_size,
    val_batch_size=val_batch_size,
)

Found 556 images in src/dataset/Motic-Human-tissues
Creating HuggingFace datasets...
Formatting training data...


Map:   0%|          | 0/444 [00:00<?, ? examples/s]

Formatting validation data...


Map:   0%|          | 0/112 [00:00<?, ? examples/s]

In [6]:
print(f"Current type classes: {dataset.type_classes}")
print(f"Current zoom classes: {dataset.zoom_classes}")
print(f"Current focus classes: {dataset.focus_classes}")

Current type classes: ['human adrenal gland', 'human bone marrow', 'human cerebelum', 'human esophagus', 'human heart section', 'human hyaline cartilage', 'human kidney', 'human liver section', 'human salivary gland', 'human small intestine', 'human spleen', 'human stratified epithelium', 'human thymus', 'human tongue', 'human tonsil tongue']
Current zoom classes: [4, 10, 20, 40]
Current focus classes: [0, 1]


In [7]:
dataset.set_processor(processor)
dataset.build_train_val_loaders()

## Set up evaluation

## inference on val data

In [8]:
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import evaluate

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")


def compute_metrics_multitask(
    predictions: list[dict], references: dict
) -> dict[str, float]:
    """Compute metrics for all three tasks: tissue_type, zoom_level, focus_quality"""
    metrics = {}

    # Extract predictions for each task
    tissue_predictions = [pred.get("tissue_type", -1) for pred in predictions]
    zoom_predictions = [pred.get("zoom_level", -1) for pred in predictions]
    focus_predictions = [pred.get("focus_quality", -1) for pred in predictions]

    # Get references for each task
    tissue_references = references["tissue_type"]
    zoom_references = references["zoom_level"]
    focus_references = references["focus_quality"]

    # Compute metrics for tissue type
    tissue_acc = accuracy_metric.compute(
        predictions=tissue_predictions, references=tissue_references
    )
    tissue_f1 = f1_metric.compute(
        predictions=tissue_predictions, references=tissue_references, average="weighted"
    )

    # Compute metrics for zoom level
    zoom_acc = accuracy_metric.compute(
        predictions=zoom_predictions, references=zoom_references
    )
    zoom_f1 = f1_metric.compute(
        predictions=zoom_predictions, references=zoom_references, average="weighted"
    )

    # Compute metrics for focus quality
    focus_acc = accuracy_metric.compute(
        predictions=focus_predictions, references=focus_references
    )
    focus_f1 = f1_metric.compute(
        predictions=focus_predictions, references=focus_references, average="weighted"
    )

    # Organize metrics by task
    metrics.update(
        {
            "tissue_type_accuracy": tissue_acc["accuracy"],
            "tissue_type_f1": tissue_f1["f1"],
            "zoom_level_accuracy": zoom_acc["accuracy"],
            "zoom_level_f1": zoom_f1["f1"],
            "focus_quality_accuracy": focus_acc["accuracy"],
            "focus_quality_f1": focus_f1["f1"],
            # Overall averages
            "overall_accuracy": (
                tissue_acc["accuracy"] + zoom_acc["accuracy"] + focus_acc["accuracy"]
            )
            / 3,
            "overall_f1": (tissue_f1["f1"] + zoom_f1["f1"] + focus_f1["f1"]) / 3,
        }
    )

    return metrics


def evaluate(model, dataset):
    model.eval()
    pt_outputs = []
    batch_idx = 0

    # Collect ground truth references for all tasks
    references = {
        "tissue_type": dataset.val_dataset["type"],
        "zoom_level": dataset.val_dataset["zoom"],
        "focus_quality": dataset.val_dataset["focus"],
    }

    for batch in tqdm(dataset.val_loader, desc="Processing validation batches"):
        # Move batch to the same device as the model
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Get actual batch size (number of samples in this batch)
        actual_batch_size = batch["input_ids"].shape[0]

        # Generate predictions for the entire batch
        outputs = model.generate(
            input_ids=batch["input_ids"],
            pixel_values=batch["pixel_values"],
            attention_mask=batch["attention_mask"],
            max_new_tokens=200,  # Increased for JSON responses
            do_sample=False,  # Set to True for sampling, False for greedy decoding
            # Add stopping criteria
            repetition_penalty=1.1,
        )

        # Decode only the newly generated tokens (exclude the input)
        input_length = batch["input_ids"].shape[1]
        generated_tokens = outputs[:, input_length:]

        # Decode the generated text
        generated_text = processor.batch_decode(
            generated_tokens, skip_special_tokens=True
        )

        # Process each sample in the current batch
        for i in range(actual_batch_size):
            # Format as expected by postprocess method
            formatted_output = {"generated_text": generated_text[i]}
            pt_outputs.append(formatted_output)

        batch_idx += 1
        # print the first generated output of the batch
        print(
            f"Batch {batch_idx} processed, first output: {pt_outputs[-1]['generated_text']}"
        )
        print("-" * 50)
    # Process predictions - now returns dict for each prediction
    pt_predictions = [dataset.postprocess([out]) for out in pt_outputs]

    print(f"\nSample predictions: {pt_predictions[:3]}")  # Show first 3 predictions
    print(f"Total predictions: {len(pt_predictions)}")

    # Compute metrics for all tasks
    pt_metrics = compute_metrics_multitask(pt_predictions, references)

    print(f"\nDetailed Metrics:")
    print(
        f"Tissue Type - Accuracy: {pt_metrics['tissue_type_accuracy']:.3f}, F1: {pt_metrics['tissue_type_f1']:.3f}"
    )
    print(
        f"Zoom Level - Accuracy: {pt_metrics['zoom_level_accuracy']:.3f}, F1: {pt_metrics['zoom_level_f1']:.3f}"
    )
    print(
        f"Focus Quality - Accuracy: {pt_metrics['focus_quality_accuracy']:.3f}, F1: {pt_metrics['focus_quality_f1']:.3f}"
    )
    print(
        f"Overall - Accuracy: {pt_metrics['overall_accuracy']:.3f}, F1: {pt_metrics['overall_f1']:.3f}"
    )

    return pt_metrics

In [11]:
base_line_metrics = evaluate(base_model, dataset)

Processing validation batches:   0%|          | 0/14 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Processing validation batches:   7%|▋         | 1/14 [00:26<05:42, 26.35s/it]

Batch 1 processed, first output: 

Based on the provided image, here's the analysis:

```json
{
  "tissue_type": "9: human small intestine",
  "zoom_level": "2: 20x",
  "focus_quality": "0: focused"
}
```
Final Answer: The final answer is $\boxed{{tissue_type: "9: human small intestine", zoom_level: "2: 20x", focus_quality: "0: focused"}} $
```json
{
  "tissue_type": "9: human small intestine",
  "zoom_level": "2: 20x",
  "focus_quality": "0
--------------------------------------------------


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Processing validation batches:   7%|▋         | 1/14 [00:48<10:35, 48.89s/it]


KeyboardInterrupt: 

## Fine tuning

In [9]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    # target_modules="all-linear",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
    ],  # Target attention layers only
    task_type="CAUSAL_LM",
    modules_to_save=[
        "lm_head",
        "embed_tokens",
    ],
)

# Apply LoRA configuration to the model
model = get_peft_model(base_model, peft_config)

In [10]:
print(f"trainable parameters: {model.print_trainable_parameters()}")

trainable params: 1,354,403,840 || all params: 5,654,483,312 || trainable%: 23.9527
trainable parameters: None


In [11]:
# Add this after loading the processor to check the image size
print(f"Processor image size: {processor.image_processor.size}")
print(f"Image processor config: {processor.image_processor}")

Processor image size: {'height': 896, 'width': 896}
Image processor config: Gemma3ImageProcessor {
  "do_convert_rgb": null,
  "do_normalize": true,
  "do_pan_and_scan": null,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "Gemma3ImageProcessor",
  "image_seq_length": 256,
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "pan_and_scan_max_num_crops": null,
  "pan_and_scan_min_crop_size": null,
  "pan_and_scan_min_ratio_to_activate": null,
  "processor_class": "Gemma3Processor",
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 896,
    "width": 896
  }
}



In [ ]:
from transformers import get_scheduler
from tqdm import tqdm
import collections


# TODO remove window accuracy
# TODO check if weighted avg

# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# -- Training parameters --
epochs = 3  # 1 for testing, 3 for full training
weight_decay = 0.001
learning_rate = 2e-5
scheduler_name = "cosine"
ratio_warmup = 0.1


optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
)

scheduler = get_scheduler(
    name=scheduler_name,
    optimizer=optimizer,
    num_warmup_steps=int(len(dataset.train_loader) * epochs * ratio_warmup),
    num_training_steps=len(dataset.train_loader) * epochs,
)


for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Training loop
    model.train()
    train_loss = 0.0
    train_steps = 0
    windows_acc = collections.deque(maxlen=5)  # Initialize the windowed accuracy queue
    pbar = tqdm(dataset.train_loader, desc="Training")
    for batch in pbar:

        # Convert messages to text format for the processor
        batch = {k: v.to(model.device) for k, v in batch.items()}

        outputs = model(**batch)

        # Only compute accuracy on non-masked labels
        predictions = torch.argmax(outputs.logits, dim=-1)
        valid_labels = batch["labels"] != -100
        if valid_labels.sum() > 0:
            batch_accuracy = (
                (predictions[valid_labels] == batch["labels"][valid_labels])
                .float()
                .mean()
                .item()
            )
        else:
            batch_accuracy = 0.0

        # Update the windowed accuracy
        windows_acc.append(batch_accuracy)
        current_windowed_acc = np.mean(windows_acc)

        # Use the model's outputs to compute the loss
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        train_loss += loss.item()
        train_steps += 1

        # Update progress bar with windowed accuracy
        pbar.set_postfix(
            {
                "loss": f"{loss.item():.4f}",
                "win_acc": f"{current_windowed_acc:.3f}",
                "lr": f"{scheduler.get_last_lr()[0]:.2e}",
            }
        )

    avg_train_loss = train_loss / train_steps
    print(f"Average Training Loss: {avg_train_loss:.4f}")

Epoch 1/3


Training:   0%|          | 0/444 [00:00<?, ?it/s]

Training: 100%|██████████| 444/444 [08:41<00:00,  1.17s/it, loss=0.0260, win_acc=0.010, lr=1.69e-05]


Average Training Loss: 0.6139
Epoch 2/3


Training:  35%|███▍      | 155/444 [03:00<05:36,  1.17s/it, loss=0.0137, win_acc=0.010, lr=1.34e-05]

In [ ]:
evaluate(model, dataset)

In [ ]:
# save the model
from time import strftime

save_path = f"src/models/medgemma-4b-it-lora-{strftime('%Y-%m-%d_%H-%M-%S')}"
model.save_pretrained(save_path)

In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
from peft import PeftModel
import torch

# Load the base model with the same configuration as during training
model_id = "google/medgemma-4b-it"

model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=model_kwargs["torch_dtype"],
    bnb_4bit_quant_storage=model_kwargs["torch_dtype"],
)

# Load the base model
base_model = AutoModelForImageTextToText.from_pretrained(model_id, **model_kwargs)

# Load the LoRA adapter on top of the base model
model = PeftModel.from_pretrained(base_model, save_path)

# Load the processor
processor = AutoProcessor.from_pretrained(model_id)
processor.tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
val_metrics = evaluate(model, dataset)

Processing validation batches:   0%|          | 0/14 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Processing validation batches:   7%|▋         | 1/14 [00:04<01:03,  4.89s/it]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Processing validation batches:  14%|█▍        | 2/14 [00:09<00:59,  4.99s/it]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Processing validation batches:  21%|██▏       | 3/14 [00:14<00:54,  4.97s/it]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. 


All predictions: [10, 7, 13, 13, 7, 10, 7, 9, 4, 5, 13, 8, 13, 5, 10, 9, 9, 10, 0, 0, 13, 13, 8, 10, 0, 0, 0, 10, 8, 9, 13, 10, 8, 8, 10, 7, 10, 0, 13, 13, 0, 7, 9, 9, 10, 8, 11, 10, 4, 8, 0, 8, 10, 7, 5, 13, 10, 0, 9, 9, 7, 13, 13, 0, 10, 9, 0, 0, 9, 8, 8, 8, 5, 5, 5, 10, 8, 10, 13, 8, 0, 13, 13, 10, 9, 10, 13, 13, 10, 9, 7, 10, 10, 10, 7, 10, 10, 5, 10, 9, 5, 8, 10, 10, 10, 11, 9, 10, 0, 4, 10, 13]
Metrics: {'accuracy': 0.6785714285714286, 'f1': 0.6322135264362155}
